/*Introduction
In this Proof of Concept (POC) stage, we aim to address the challenge of comparing datasets and establishing mappings between a legacy data system and a modern data system. This document provides a glimpse of how we can achieve dataset comparison and produce a mapping between legacy data and modern datasets by leveraging vector embeddings ,Generative AI and advanced query techniques with Snowflake and Snowflake cortex AI.

Problem Statement
The core problem we are tackling involves comparing two datasets: one from a legacy system and the other from a modern system. The goal is to identify and map corresponding fields between these datasets to ensure data consistency, accuracy, and usability in the modern system.

Approach
We will create two schemas, one for the modern dataset (Deposit_Accounts_Onestream) and one for the legacy dataset (Legacy_Deposit_Accounts). Using these schemas, we will create metadata tables to store field descriptions. We will then use vector embeddings to compare the fields from both datasets and identify the top three most similar fields for each field in the modern dataset.

This POC demonstrates the steps and processes involved in creating these mappings and highlights the potential benefits of using vector embeddings for dataset comparison.*/

In [ ]:
--Create Your Own Database  and Schema and 
CREATE DATABASE SMART_AI_MAPPER;
CREATE SCHEMA SMART_AI_MAPPER_TOOL;

USE SMART_AI_MAPPER.SMART_AI_MAPPER_TOOL;


-- Create 2 new tables named Deposit_Accounts_Modernised  and Deposit_Accounts_Legacy to store customer and account information, this table should hold the sample data of the table.Whereas You can modify the scripts create tables with sample data with key columns for Modernised and Legacy dataset.

In [ ]:
CREATE OR REPLACE TABLE Deposit_Accounts_Modernised(
    Customer_ID VARCHAR(50) ,
    First_Name VARCHAR(100),
    Last_Name VARCHAR(100),
    Date_of_Birth DATE,
    SSN VARCHAR(11),
    Email VARCHAR(100),
    Phone_Number VARCHAR(15),
    Address VARCHAR(200),
    City VARCHAR(50),
    State VARCHAR(50),
    Zip_Code VARCHAR(10),
    Account_Number VARCHAR(50) PRIMARY KEY,
    Account_Type VARCHAR(50),
    Account_Balance DECIMAL(18, 2),
    Interest_Rate DECIMAL(5, 2),
    Account_Status VARCHAR(20),
    Date_Opened DATE,
    Date_Closed DATE,
    Branch_ID VARCHAR(50),
    Last_Transaction_Date DATE,
    KYC_Status VARCHAR(20),
    Risk_Level VARCHAR(20),
    Marketing_Opt_In BOOLEAN,
    Created_Timestamp TIMESTAMP,
    Updated_Timestamp TIMESTAMP
);

INSERT INTO Deposit_Accounts_Modernised (
    Customer_ID, First_Name, Last_Name, Date_of_Birth, SSN, Email, Phone_Number,
    Address, City, State, Zip_Code, Account_Number, Account_Type, Account_Balance,
    Interest_Rate, Account_Status, Date_Opened, Date_Closed, Branch_ID, Last_Transaction_Date,
    KYC_Status, Risk_Level, Marketing_Opt_In, Created_Timestamp, Updated_Timestamp
) VALUES
(
    'CUST001', 'John', 'Doe', '1980-01-15', '123-45-6789', 'john.doe@example.com', '555-1234',
    '123 Maple St', 'Springfield', 'IL', '62701', 'ACC12345', 'Savings', 1000.50,
    1.50, 'Active', '2023-01-01', NULL, 'BR001', '2024-07-25',
    'Verified', 'Low', TRUE, '2023-01-01 09:00:00', '2024-07-25 10:00:00'
),
(
    'CUST002', 'Jane', 'Smith', '1990-02-20', '987-65-4321', 'jane.smith@example.com', '555-5678',
    '456 Oak St', 'Greenfield', 'WI', '53220', 'ACC23456', 'Checking', 2500.75,
    0.75, 'Active', '2022-05-15', NULL, 'BR002', '2024-07-24',
    'Verified', 'Medium', FALSE, '2022-05-15 10:30:00', '2024-07-24 15:00:00'
),
(
    'CUST003', 'Michael', 'Johnson', '1975-03-10', '456-78-9123', 'michael.johnson@example.com', '555-9876',
    '789 Pine St', 'Lakeview', 'MN', '55401', 'ACC34567', 'Savings', 1500.00,
    1.25, 'Active', '2023-07-01', NULL, 'BR003', '2024-07-23',
    'Pending', 'High', TRUE, '2023-07-01 14:00:00', '2024-07-23 13:00:00'
),
(
    'CUST004', 'Emily', 'Davis', '1985-04-25', '321-54-9876', 'emily.davis@example.com', '555-3456',
    '321 Cedar St', 'Hilltown', 'OH', '44101', 'ACC45678', 'Checking', 3200.00,
    0.50, 'Active', '2021-11-20', NULL, 'BR004', '2024-07-22',
    'Verified', 'Low', FALSE, '2021-11-20 08:45:00', '2024-07-22 09:15:00'
),
(
    'CUST005', 'David', 'Williams', '1995-06-30', '654-32-1987', 'david.williams@example.com', '555-6543',
    '654 Birch St', 'Mapleton', 'PA', '17055', 'ACC56789', 'Savings', 500.00,
    1.75, 'Inactive', '2022-12-10', '2023-12-10', 'BR005', '2023-12-09',
    'Verified', 'Medium', TRUE, '2022-12-10 11:00:00', '2023-12-10 11:00:00'
);

CREATE OR REPLACE TABLE Deposit_Accounts_Legacy (
    Cust_ID VARCHAR(50) ,
    F_Name VARCHAR(100),
    L_Name VARCHAR(100),
    DOB DATE,
    Social_Security_Number VARCHAR(11),
    Email_Address VARCHAR(100),
    Phone VARCHAR(15),
    Addr VARCHAR(200),
    City VARCHAR(50),
    State VARCHAR(50),
    Postal_Code VARCHAR(10),
    Acct_Num VARCHAR(50) PRIMARY KEY,
    Acct_Type VARCHAR(50),
    Acct_Balance DECIMAL(18, 2),
    Interest DECIMAL(5, 2),
    Acct_Status VARCHAR(20),
    Open_Date DATE,
    Close_Date DATE,
    Branch_ID VARCHAR(50),
    Last_Trans_Date DATE,
    KYC VARCHAR(20),
    Risk VARCHAR(20),
    Marketing_Consent BOOLEAN,
    Creation_Timestamp TIMESTAMP,
    Modification_Timestamp TIMESTAMP,
    Middle_Name VARCHAR(100),
    Nationality VARCHAR(50),
    Employment_Status VARCHAR(50)
);

INSERT INTO Deposit_Accounts_Legacy (
    Cust_ID, F_Name, L_Name, DOB, Social_Security_Number, Email_Address, Phone,
    Addr, City, State, Postal_Code, Acct_Num, Acct_Type, Acct_Balance, Interest,
    Acct_Status, Open_Date, Close_Date, Branch_ID, Last_Trans_Date, KYC, Risk,
    Marketing_Consent, Creation_Timestamp, Modification_Timestamp, Middle_Name,
    Nationality, Employment_Status
) VALUES
(
    'CUST001', 'John', 'Doe', '1980-01-15', '123-45-6789', 'john.doe@example.com', '555-1234',
    '123 Maple St', 'Springfield', 'IL', '62701', 'ACC12345', 'Savings', 1000.50, 1.5000,
    'Active', '2023-01-01', NULL, 'BR001', '2024-07-25', 'Verified', 'Low', TRUE,
    '2023-01-01 09:00:00', '2024-07-25 10:00:00', 'Michael', 'USA', 'Employed'
),
(
    'CUST002', 'Jane', 'Smith', '1990-02-20', '987-65-4321', 'jane.smith@example.com', '555-5678',
    '456 Oak St', 'Greenfield', 'WI', '53220', 'ACC23456', 'Checking', 2500.7500, 0.7500,
    'Active', '2022-05-15', NULL, 'BR002', '2024-07-24', 'Verified', 'Medium', FALSE,
    '2022-05-15 10:30:00', '2024-07-24 15:00:00', 'Ann', 'USA', 'Self-employed'
),
(
    'CUST003', 'Michael', 'Johnson', '1975-03-10', '456-78-9123', 'michael.johnson@example.com', '555-9876',
    '789 Pine St', 'Lakeview', 'MN', '55401', 'ACC34567', 'Savings', 1500.00, 1.2500,
    'Active', '2023-07-01', NULL, 'BR003', '2024-07-23', 'Pending', 'High', TRUE,
    '2023-07-01 14:00:00', '2024-07-23 13:00:00', 'Edward', 'Canada', 'Unemployed'
),
(
    'CUST004', 'Emily', 'Davis', '1985-04-25', '321-54-9876', 'emily.davis@example.com', '555-3456',
    '321 Cedar St', 'Hilltown', 'OH', '44101', 'ACC45678', 'Checking', 3200.00, 0.5000,
    'Active', '2021-11-20', NULL, 'BR004', '2024-07-22', 'Verified', 'Low', FALSE,
    '2021-11-20 08:45:00', '2024-07-22 09:15:00', 'Marie', 'UK', 'Part-time'
),
(
    'CUST005', 'David', 'Williams', '1995-06-30', '654-32-1987', 'david.williams@example.com', '555-6543',
    '654 Birch St', 'Mapleton', 'PA', '17055', 'ACC56789', 'Savings', 500.00, 1.7500,
    'Inactive', '2022-12-10', '2023-12-10', 'BR005', '2023-12-09', 'Verified', 'Medium', TRUE,
    '2022-12-10 11:00:00', '2023-12-10 11:00:00', 'Charles', 'Australia', 'Retired'
);

Next Step is to insert the metadata and field level description of modernised dataset, which will be used to compare modernised table field level - Legacy table field level using vector embeddings.

In [ ]:
CREATE OR REPLACE TABLE Attribute_level_Metadata (
    Table_Name VARCHAR(100), 
    Field_Name VARCHAR(100),
    Description VARCHAR(1000)
);

DELETE FROM Attribute_level_Metadata WHERE Table_Name='Deposit_Accounts_Modernised_Metadata';

INSERT INTO Attribute_level_Metadata (Table_Name, Field_Name, Description) VALUES
('Deposit_Accounts_Modernised_Metadata', 'Customer_ID', 'A unique identifier for each customer. This field ensures that each customer can be distinctly recognized and associated with their corresponding data.'),
('Deposit_Accounts_Modernised_Metadata', 'First_Name', 'The first name of the customer. Used for identification and communication purposes.'),
('Deposit_Accounts_Modernised_Metadata', 'Last_Name', 'The last name of the customer. Used for identification and communication purposes.'),
('Deposit_Accounts_Modernised_Metadata', 'Date_of_Birth', 'The customer\'s date of birth. This is non-public information used for verification and age-related services.'),
('Deposit_Accounts_Modernised_Metadata', 'SSN', 'The customer\'s Social Security Number. This is highly sensitive non-public information used for identification and fraud prevention.'),
('Deposit_Accounts_Modernised_Metadata', 'Email', 'The customer\'s email address. This non-public information is used for communication and notification purposes.'),
('Deposit_Accounts_Modernised_Metadata', 'Phone_Number', 'The customer\'s phone number. This non-public information is used for communication and notification purposes.'),
('Deposit_Accounts_Modernised_Metadata', 'Address', 'The residential address of the customer. This is used for verification, correspondence, and service delivery.'),
('Deposit_Accounts_Modernised_Metadata', 'City', 'The city where the customer resides. This is part of the customer\'s address information.'),
('Deposit_Accounts_Modernised_Metadata', 'State', 'The state where the customer resides. This is part of the customer\'s address information.'),
('Deposit_Accounts_Modernised_Metadata', 'Zip_Code', 'The zip code of the customer\'s residence. This is part of the customer\'s address information.'),
('Deposit_Accounts_Modernised_Metadata', 'Account_Number', 'A unique identifier for each deposit account. This ensures each account can be distinctly recognized and managed.'),
('Deposit_Accounts_Modernised_Metadata', 'Account_Type', 'The type of deposit account (e.g., Savings, Checking). This categorizes the account based on its purpose and features.'),
('Deposit_Accounts_Modernised_Metadata', 'Account_Balance', 'The current balance in the deposit account. This reflects the available funds in the account.'),
('Deposit_Accounts_Modernised_Metadata', 'Interest_Rate', 'The interest rate applicable to the deposit account. This indicates the percentage of interest earned or charged on the account balance.'),
('Deposit_Accounts_Modernised_Metadata', 'Account_Status', 'The status of the account (e.g., Active, Closed). This indicates whether the account is currently operational or has been closed.'),
('Deposit_Accounts_Modernised_Metadata', 'Date_Opened', 'The date when the account was opened. This records the start date of the account.'),
('Deposit_Accounts_Modernised_Metadata', 'Date_Closed', 'The date when the account was closed, if applicable. This records the end date of the account.'),
('Deposit_Accounts_Modernised_Metadata', 'Branch_ID', 'The identifier for the branch where the account was opened. This links the account to its originating branch.'),
('Deposit_Accounts_Modernised_Metadata', 'Last_Transaction_Date', 'The date of the last transaction in the account. This indicates the most recent activity in the account.'),
('Deposit_Accounts_Modernised_Metadata', 'KYC_Status', 'The Know Your Customer status (e.g., Verified, Pending). This indicates whether the customer has been verified according to KYC regulations.'),
('Deposit_Accounts_Modernised_Metadata', 'Risk_Level', 'The risk level assigned to the customer/account. This is used for monitoring and managing potential risks.'),
('Deposit_Accounts_Modernised_Metadata', 'Marketing_Opt_In', 'Indicates if the customer has opted in for marketing communications. This shows the customer\'s preference for receiving promotional material.'),
('Deposit_Accounts_Modernised_Metadata', 'Created_Timestamp', 'The timestamp when the record was created. This records the exact time the record was added to the database.'),
('Deposit_Accounts_Modernised_Metadata', 'Updated_Timestamp', 'The timestamp when the record was last updated. This records the last modification time for the record.');

DELETE FROM Attribute_level_Metadata WHERE Table_Name='Deposit_Accounts_Legacy_Metadata';

INSERT INTO Attribute_level_Metadata
(Table_Name,Field_Name, Description) VALUES
('Deposit_Accounts_Legacy_Metadata','Cust_ID', 'Unique identifier for each customer.'),
('Deposit_Accounts_Legacy_Metadata','F_Name', 'Customer\'s first name.'),
('Deposit_Accounts_Legacy_Metadata','L_Name', 'Customer\'s last name.'),
('Deposit_Accounts_Legacy_Metadata','DOB', 'Customer\'s date of birth (Non-Public Information).'),
('Deposit_Accounts_Legacy_Metadata','Social_Security_Number', 'Customer\'s Social Security Number (Non-Public Information, Payment Card Information).'),
('Deposit_Accounts_Legacy_Metadata','Email_Address', 'Customer\'s email address (Non-Public Information).'),
('Deposit_Accounts_Legacy_Metadata','Phone', 'Customer\'s phone number (Non-Public Information).'),
('Deposit_Accounts_Legacy_Metadata','Addr', 'Customer\'s residential address.'),
('Deposit_Accounts_Legacy_Metadata','City', 'City of the customer\'s residence.'),
('Deposit_Accounts_Legacy_Metadata','State', 'State of the customer\'s residence.'),
('Deposit_Accounts_Legacy_Metadata','Postal_Code', 'Zip code of the customer\'s residence.'),
('Deposit_Accounts_Legacy_Metadata','Acct_Num', 'Unique identifier for each deposit account.'),
('Deposit_Accounts_Legacy_Metadata','Acct_Type', 'Type of deposit account (e.g., Savings, Checking).'),
('Deposit_Accounts_Legacy_Metadata','Acct_Balance', 'Current balance in the deposit account.'),
('Deposit_Accounts_Legacy_Metadata','Interest', 'Interest rate applicable to the deposit account.'),
('Deposit_Accounts_Legacy_Metadata','Acct_Status', 'Status of the account (e.g., Active, Closed).'),
('Deposit_Accounts_Legacy_Metadata','Open_Date', 'Date when the account was opened.'),
('Deposit_Accounts_Legacy_Metadata','Close_Date', 'Date when the account was closed (if applicable).'),
('Deposit_Accounts_Legacy_Metadata','Branch_ID', 'Identifier for the branch where the account was opened.'),
('Deposit_Accounts_Legacy_Metadata','Last_Trans_Date', 'Date of the last transaction in the account.'),
('Deposit_Accounts_Legacy_Metadata','KYC', 'Know Your Customer status (e.g., Verified, Pending).'),
('Deposit_Accounts_Legacy_Metadata','Risk', 'Risk level assigned to the customer/account.'),
('Deposit_Accounts_Legacy_Metadata','Marketing_Consent', 'Indicates if the customer has opted in for marketing communications.'),
('Deposit_Accounts_Legacy_Metadata','Creation_Timestamp', 'Timestamp when the record was created.'),
('Deposit_Accounts_Legacy_Metadata','Modification_Timestamp', 'Timestamp when the record was last updated.'),
('Deposit_Accounts_Legacy_Metadata','Middle_Name', 'Customer\'s middle name.'),
('Deposit_Accounts_Legacy_Metadata','Nationality', 'Customer\'s nationality.'),
('Deposit_Accounts_Legacy_Metadata','Employment_Status', 'Customer\'s employment status.');

In [ ]:
--Check Sampling of Meata Data

SELECT TOP 2 * FROM Attribute_level_Metadata where table_name='Deposit_Accounts_Modernised_Metadata' 

UNION ALL

SELECT TOP 2 * FROM Attribute_level_Metadata where table_name='Deposit_Accounts_Legacy_Metadata' ;


In [ ]:
--Check Sampling of both Table Data of Modernised/Legacy

SELECT TOP 2 * FROM Deposit_Accounts_Modernised


In [ ]:
SELECT TOP 2 * FROM Deposit_Accounts_Legacy;

In [ ]:
---DDL TO CREATE VECTOR EMBEDDINGS TABLE
CREATE or REPLACE TABLE FIELD_LEVEL_METADATA_WITH_VEC_EMB (
    Table_Name VARCHAR(100), 
    Field_Name VARCHAR(100),
    Description VARCHAR(1000),
	VEC_EMB_DESC VECTOR(FLOAT, 768)
);

In [ ]:
--DERIVE THE VECTOR EMBEDDING USING LLM EMBEDDING MODEL snowflake-arctic-embed-m


INSERT INTO FIELD_LEVEL_METADATA_WITH_VEC_EMB
SELECT 
Table_Name,
Field_Name,
CONCAT(Field_Name,'-',Description) AS Description,
SNOWFLAKE.CORTEX.EMBED_TEXT_768('snowflake-arctic-embed-m',Description) AS VEC_EMB_DESC FROM Attribute_level_Metadata
WHERE Table_Name IN ('Deposit_Accounts_Modernised_Metadata',
'Deposit_Accounts_Legacy_Metadata')

In [ ]:
SELECT TOP 2 * FROM FIELD_LEVEL_METADATA_WITH_VEC_EMB
WHERE Table_Name ='Deposit_Accounts_Modernised_Metadata'
UNION ALL
SELECT TOP 2 * FROM FIELD_LEVEL_METADATA_WITH_VEC_EMB
WHERE Table_Name ='Deposit_Accounts_Legacy_Metadata'

In [ ]:
CREATE or REPLACE TABLE TOP_3_SIMILAR_FIELDS_FROM_VEC_EMB (
    Vector_Mapping_For_Table_Name VARCHAR(1000), 
    Modernised_Table_Field_Name VARCHAR(100),
	TOP_SIMILARITIES_LEGACY_FIELDS VARCHAR(16777216)
);

In [ ]:
DELETE FROM TOP_3_SIMILAR_FIELDS_FROM_VEC_EMB
WHERE 
Vector_Mapping_For_Table_Name=' DEPOSIT_ACCOUNTS_MODERNISED_METADATA_MAPPED_DEPOSIT_ACCOUNTS_LEGACY_METADATA';

INSERT INTO TOP_3_SIMILAR_FIELDS_FROM_VEC_EMB
SELECT 
        UPPER(Vector_Mapping_For_Table_Name) AS Vector_Mapping_For_Table_Name,
        MODERNISED_FIELD_NAME ,
        top_similarities
    FROM (
        SELECT 
            Vector_Mapping_For_Table_Name,
            MODERNISED_FIELD_NAME,
            LISTAGG(CONCAT(LEGACY_FIELD_NAME,'-Similarity',similarity,'|')) within group 
            (ORDER BY similarity DESC) AS top_similarities
        FROM (
            SELECT 
            Vector_Mapping_For_Table_Name,
                MODERNISED_FIELD_NAME,
                LEGACY_FIELD_NAME,
                similarity,
                ROW_NUMBER() OVER (PARTITION BY Vector_Mapping_For_Table_Name,MODERNISED_FIELD_NAME ORDER BY similarity DESC) AS rank
            FROM (
                SELECT 
                CONCAT(TRIM(MODERNISED.Table_Name),'_Mapped_',
                TRIM(LEGACY.Table_Name))        
                 AS Vector_Mapping_For_Table_Name,
                    MODERNISED.FIELD_NAME AS MODERNISED_FIELD_NAME,
                    LEGACY.FIELD_NAME AS LEGACY_FIELD_NAME,
                    VECTOR_COSINE_SIMILARITY(MODERNISED.VEC_EMB_DESC, LEGACY.VEC_EMB_DESC) AS similarity
                FROM 
                (
                SELECT * FROM FIELD_LEVEL_METADATA_WITH_VEC_EMB
                WHERE Table_Name='Deposit_Accounts_Modernised_Metadata'
                )MODERNISED
                CROSS JOIN                 (
                SELECT * FROM FIELD_LEVEL_METADATA_WITH_VEC_EMB
                WHERE Table_Name='Deposit_Accounts_Legacy_Metadata'
                )LEGACY 
                
            ) AS Similarity_CTE
        ) AS Ranked_Similarities
        WHERE rank <= 3
        GROUP BY Vector_Mapping_For_Table_Name,MODERNISED_FIELD_NAME
    ) AS Top_Similarities;

In [ ]:
SELECT TOP 3 * FROM TOP_3_SIMILAR_FIELDS_FROM_VEC_EMB;